In [0]:
df = spark.read.load("/Volumes/dai/phase2/silver/final_data_with_weights/")
display(df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,class_weight
17420,598.8300000000002,30,265,2011-10-20,0,0.6302969155376189
16552,379.72999999999996,17,219,2011-04-11,0,0.6302969155376189
17572,226.75,12,95,2011-09-29,0,0.6302969155376189
15350,115.65,5,51,2010-12-01,0,0.6302969155376189
12921,16389.740000000016,741,9454,2011-12-06,1,2.418694690265487
13090,8689.390000000003,161,2194,2011-12-01,1,2.418694690265487
14135,4690.31,134,3850,2011-12-08,1,2.418694690265487
12915,363.65,22,93,2011-07-14,0,0.6302969155376189
17685,3191.5299999999993,130,1956,2011-11-23,1,2.418694690265487
17581,10736.109999999991,452,5861,2011-12-09,1,2.418694690265487


In [0]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

## Creating a single input vector feature 

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["total_spent", "total_transactions", "total_quantity"],
    outputCol="features"
)

train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

In [0]:
display(train_df.head(10))
display(test_df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,class_weight,features
null,1447682.1199996774,135080,269562,2011-12-09,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1447682.1199996774"",""135080.0"",""269562.0""]}"
12346,0.0,2,0,2011-01-18,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""2.0"",""0.0""]}"
12347,4309.999999999997,182,2458,2011-12-07,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""4309.999999999997"",""182.0"",""2458.0""]}"
12348,1797.24,31,2341,2011-09-25,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1797.24"",""31.0"",""2341.0""]}"
12349,1757.55,73,631,2011-11-21,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1757.55"",""73.0"",""631.0""]}"
12350,334.40000000000003,17,197,2011-02-02,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""334.40000000000003"",""17.0"",""197.0""]}"
12352,1545.4100000000005,95,470,2011-11-03,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1545.4100000000005"",""95.0"",""470.0""]}"
12353,89.0,4,20,2011-05-19,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""89.0"",""4.0"",""20.0""]}"
12355,459.4,13,240,2011-05-09,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""459.4"",""13.0"",""240.0""]}"
12356,2811.4300000000007,59,1591,2011-11-17,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""2811.4300000000007"",""59.0"",""1591.0""]}"


CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,class_weight,features
12354,1079.4,58,530,2011-04-21,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1079.4"",""58.0"",""530.0""]}"
12357,6207.669999999996,131,2708,2011-11-06,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""6207.669999999996"",""131.0"",""2708.0""]}"
12359,6245.530000000006,254,1612,2011-12-02,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""6245.530000000006"",""254.0"",""1612.0""]}"
12370,3545.689999999998,167,2353,2011-10-19,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""3545.689999999998"",""167.0"",""2353.0""]}"
12372,1298.0400000000002,52,794,2011-09-29,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1298.0400000000002"",""52.0"",""794.0""]}"
12373,364.6,14,197,2011-02-01,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""364.6"",""14.0"",""197.0""]}"
12374,742.93,33,342,2011-11-14,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""742.93"",""33.0"",""342.0""]}"
12377,1628.1199999999997,77,944,2011-01-28,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1628.1199999999997"",""77.0"",""944.0""]}"
12380,2720.56,105,1126,2011-11-18,1,2.418694690265487,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""2720.56"",""105.0"",""1126.0""]}"
12386,401.90000000000003,10,354,2011-01-06,0,0.6302969155376189,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""401.90000000000003"",""10.0"",""354.0""]}"


In [0]:
train_df.write.format("delta").mode("overwrite").save("/Volumes/dai/phase2/silver/train_df")
test_df.write.format("delta").mode("overwrite").save("/Volumes/dai/phase2/silver/test_df")

---

# Training the Logistic Regression

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="is_high_valued",
    weightCol="class_weight"
)

lr_model = lr.fit(train_df)

In [0]:
lr_predictions = lr_model.transform(test_df)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="is_high_valued",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)

lr_auc = evaluator.evaluate(lr_predictions)
print("Logistic Regression AUC:", lr_auc)

Logistic Regression AUC: 1.0


---

# Now, Training Random Forest Classifier

In [0]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="is_high_valued",
    weightCol="class_weight",
    numTrees=30
)

rf_model = rf.fit(train_df)

## Testing the Random Forest Model

In [0]:
rf_predictions = rf_model.transform(test_df)

rf_auc = evaluator.evaluate(rf_predictions)
print("Random Forest AUC:", rf_auc)

Random Forest AUC: 0.9999510631530011


---

# AUC Score of Logistic and Random Forest:

1. For Logistic Regression Model, the AUC-ROC score is `1.0` indicating the model is 100% accurate
2. For Random Forest Model, the AUC-ROC score is `0.9999` indicating the model is 99.99% accurate